!py -m pip install nltk

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
# from nltk.stem import Lemmatizer

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [2]:
df=pd.read_csv('SMSSpamCollection.txt',sep='\t',names=['label','message'])
df.shape

(5572, 2)

In [3]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
for i in range(df.shape[0]):
    df.loc[i,'message']=re.sub('[^a-zA-Z]',' ',df.loc[i,'message'])

In [5]:
df.head()

,label,message
0,ham,Go until jurong point crazy Available only ...
1,ham,Ok lar Joking wif u oni
2,spam,Free entry in a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor U c already then say
4,ham,Nah I don t think he goes to usf he lives aro...


In [6]:
ps=PorterStemmer()
for i in range(df.shape[0]):
    words=df.loc[i,'message'].lower().split()
    a=[]
    for word in words:
        if not word in stopwords.words('english'):
            a.append(ps.stem(word=word))
    df.loc[i,'message']=' '.join(a)
df.head()

,label,message
0,ham,go jurong point crazi avail bugi n great world...
1,ham,ok lar joke wif u oni
2,spam,free entri wkli comp win fa cup final tkt st m...
3,ham,u dun say earli hor u c alreadi say
4,ham,nah think goe usf live around though


In [7]:
Y=pd.get_dummies(df['label'],drop_first=True)
Y.head()

,spam
0,False
1,False
2,True
3,False
4,False


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [11]:
cv=CountVectorizer(max_features=25)
X=cv.fit_transform(df['message'])
X=X.toarray()

In [13]:
X=pd.DataFrame(X)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=4)
model=MultinomialNB().fit(X_train,Y_train)
y_pred=model.predict(X_test)

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(Y_test,y_pred),columns=['Predicted Genuine', 'Predicted Spam'],index=['Actual Genuine','Actual Spam'])

,Predicted Genuine,Predicted Spam
Actual Genuine,1169,15
Actual Spam,113,96


In [33]:
y_pred_proba=model.predict_proba(X_test)
y_pred_proba

array([[0.98810002, 0.01189998],
       [0.87126107, 0.12873893],
       [0.98621784, 0.01378216],
       ...,
       [0.9856851 , 0.0143149 ],
       [0.99787239, 0.00212761],
       [0.87126107, 0.12873893]])

In [40]:
Y_pred=[0 if i<0.5 else 1 for i in y_pred_proba[:,1]]
pd.DataFrame(confusion_matrix(Y_test,Y_pred),columns=['Predicted Genuine', 'Predicted Spam'],index=['Actual Genuine','Actual Spam'])

,Predicted Genuine,Predicted Spam
Actual Genuine,1169,15
Actual Spam,113,96


In [41]:
Y_pred=[0 if i<0.4 else 1 for i in y_pred_proba[:,1]]
pd.DataFrame(confusion_matrix(Y_test,Y_pred),columns=['Predicted Genuine', 'Predicted Spam'],index=['Actual Genuine','Actual Spam'])

,Predicted Genuine,Predicted Spam
Actual Genuine,1160,24
Actual Spam,104,105


In [42]:
Y_pred=[0 if i<0.6 else 1 for i in y_pred_proba[:,1]]
pd.DataFrame(confusion_matrix(Y_test,Y_pred),columns=['Predicted Genuine', 'Predicted Spam'],index=['Actual Genuine','Actual Spam'])

,Predicted Genuine,Predicted Spam
Actual Genuine,1173,11
Actual Spam,122,87


In [43]:
Y_pred=[0 if i<0.3 else 1 for i in y_pred_proba[:,1]]
pd.DataFrame(confusion_matrix(Y_test,Y_pred),columns=['Predicted Genuine', 'Predicted Spam'],index=['Actual Genuine','Actual Spam'])

,Predicted Genuine,Predicted Spam
Actual Genuine,1127,57
Actual Spam,43,166


In [44]:
Y_pred=[0 if i<0.7 else 1 for i in y_pred_proba[:,1]]
pd.DataFrame(confusion_matrix(Y_test,Y_pred),columns=['Predicted Genuine', 'Predicted Spam'],index=['Actual Genuine','Actual Spam'])

,Predicted Genuine,Predicted Spam
Actual Genuine,1175,9
Actual Spam,131,78
